In [1]:
import pandas as pd
import numpy as np
import itertools
from Bio import SeqIO
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
import math
import gzip
import os

# APOBEC mutagenesis in MPXV
## Table of contents <a name="TOC"></a>
1. [Downloading data](#dwnl)
2. [Plots with mutations by allele for VCF files](#vcf_allele_plots)
3. [Comparison C>T mutations by motifs with other types of mutations](#compare)
4. [Simulation](#simulation)
    1. [Visualization of shares of real and simulated positions with mutations from potential ones](#shares)
    2. [Visualization of real and simulated number of reads for targets positions by replica and time](#sim_Nreads)
    3. [Get positions with the greatest and the least number of reads in real sample](#greatest_pos)

## Downloading Oxford Nanopore long-read RNA-seq data from project PRJEB56841
source - DOI: 10.1038/s41597-023-02149-4

<a name='dwnl'></a>
[Return to Table of Contents](#TOC)

In [4]:
# list with samples paths for downloading (PRJEB56841 project)
df = pd.read_csv('filereport_read_run_PRJEB56841_tsv.txt', sep='\t')
fastq_ftp = df.fastq_ftp.to_list()
fastq_ftp = ["ftp://"+i for i in fastq_ftp]
' '.join(fastq_ftp)

'ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR105/081/ERR10543481/ERR10543481.fastq.gz ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR105/074/ERR10513574/ERR10513574.fastq.gz ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR105/019/ERR10550019/ERR10550019.fastq.gz ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR105/021/ERR10550021/ERR10550021.fastq.gz ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR105/022/ERR10550022/ERR10550022.fastq.gz ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR105/024/ERR10550024/ERR10550024.fastq.gz ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR105/023/ERR10550023/ERR10550023.fastq.gz ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR105/031/ERR10550031/ERR10550031.fastq.gz ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR105/025/ERR10550025/ERR10550025.fastq.gz ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR105/033/ERR10550033/ERR10550033.fastq.gz ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR109/007/ERR10963107/ERR10963107.fastq.gz ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR105/028/ERR10550028/ERR10550028.fastq.gz ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR109/010/ERR10

script for downloading samples and mapping to the genome: scripts/RNAseq_MPXV_downl_process.sh

script for searching SNP in BAM files: scripts/clair3.sh

## Plots with mutations by allele for VCF files 

<a name='vcf_allele_plots'></a>
[Return to Table of Contents](#TOC)

In [20]:
import os

fig_list = ['Nreads', 'freq']
align_method = 'minimap2'

##min and max for variant frequency
min_VF = 0.001
max_VF = 0.9

##read genome file
###to find position in genome: str(genome_dict['NC_045512.2'].seq)
genome_file = open(r"\NC_063383.1.fasta")
genome_dict = SeqIO.to_dict(SeqIO.parse(genome_file, "fasta"))

##palette for pictures
colors_list = ['#C6878F', '#3D405B', '#81B29A', '#E07A5F', '#5F797B', '#F2CC8F']

#samples description: replica, time and file type
## table source - DOI: 10.1038/s41597-023-02149-4 
samples_features = pd.read_excel(r"\41597_2023_2149_MOESM2_ESM.xlsx", skiprows=2)
samples_features = samples_features.fillna(method='ffill', axis=0)
samples_features.head()

samples_features_dict = samples_features.set_index('Run accession')['Library Name'].to_dict()
print(samples_features_dict)

{'ERR10550019': 'MPOX_inf_1h_A_pass', 'ERR10963107': 'MPOX_inf_1h_A_pass', 'ERR10550020': 'MPOX_inf_1h_B_pass', 'ERR10963108': 'MPOX_inf_1h_B_pass', 'ERR10550021': 'MPOX_inf_1h_C_pass', 'ERR10963109': 'MPOX_inf_1h_C_pass', 'ERR10550022': 'MPOX_inf_2h_A_pass', 'ERR10963110': 'MPOX_inf_2h_A_pass', 'ERR10550023': 'MPOX_inf_2h_B_pass', 'ERR10963111': 'MPOX_inf_2h_B_pass', 'ERR10550024': 'MPOX_inf_2h_C_pass', 'ERR10963112': 'MPOX_inf_2h_C_pass', 'ERR10550025': 'MPOX_inf_4h_A_pass', 'ERR10963113': 'MPOX_inf_4h_A_pass', 'ERR10550026': 'MPOX_inf_4h_B_pass', 'ERR10963114': 'MPOX_inf_4h_B_pass', 'ERR10550027': 'MPOX_inf_4h_C_pass', 'ERR10963115': 'MPOX_inf_4h_C_pass', 'ERR10550028': 'MPOX_inf_6h_A_pass', 'ERR10963116': 'MPOX_inf_6h_A_pass', 'ERR10550029': 'MPOX_inf_6h_B_pass', 'ERR10963117': 'MPOX_inf_6h_B_pass', 'ERR10550030': 'MPOX_inf_6h_C_pass', 'ERR10963118': 'MPOX_inf_6h_C_pass', 'ERR10543479': 'MPOX_inf_12h_A_pass', 'ERR10963119': 'MPOX_inf_12h_A_pass', 'ERR10543480': 'MPOX_inf_12h_B_pass

In [24]:
#function to rounding up top position
def round_up(n, decimals=0):
    multiplier = 10**decimals
    return math.ceil(n * multiplier) / multiplier

def get_plot_vcf(vcf_file, ID, feature_sample, method=align_method, min_vf=min_VF, max_vf=max_VF, genome=str(genome_dict['NC_063383.1'].seq), colors=colors_list):
    
    ##read vcf file
    ##skip description in vcf file
    with open(vcf_file, 'r') as f:
        reader=f.readlines()
        row = 0
        while reader[row].startswith('##') == True:
            row += 1
    df_vcf = pd.read_csv(vcf_file, skiprows=row, sep='\t')
    df_vcf = df_vcf.dropna()
    
    ##make dataframe from vcf file
    ###list with all allele variants of 3 nucleotides
    A = ['A'.join(i) for i in itertools.product('ATCG', repeat=2)]
    T = ['T'.join(i) for i in itertools.product('ATCG', repeat=2)]
    C = ['C'.join(i) for i in itertools.product('ATCG', repeat=2)]
    G = ['G'.join(i) for i in itertools.product('ATCG', repeat=2)]
    allele_list = A + T + C + G

    ###dataframe for counting reads number for mutations
    df_Nreads = pd.DataFrame(columns=['ref_nucl', 'allele', 'A', 'T', 'C', 'G'])
    df_Nreads['ref_nucl'] = ['A']*16 + ['T']*16 + ['C']*16 + ['G']*16
    df_Nreads['allele'] = allele_list
    df_Nreads = df_Nreads.set_index('allele')
    df_Nreads.fillna(0, inplace=True)
    ###replace cells with no mutation (like A>A) with NaN
    for nucl in ['A', 'T', 'C', 'G']:
        df_Nreads.loc[df_Nreads['ref_nucl'] == nucl, nucl] = np.nan
    
    ###dataframe for counting allele's frequencies for mutations
    df_freq = pd.DataFrame(columns=['ref_nucl', 'allele', 'A', 'T', 'C', 'G'])
    df_freq['ref_nucl'] = ['A']*16 + ['T']*16 + ['C']*16 + ['G']*16
    df_freq['allele'] = allele_list
    df_freq = df_freq.set_index('allele')
    df_freq.fillna(0, inplace=True)
    ###replace cells with no mutation (like A>A) with NaN
    for nucl in ['A', 'T', 'C', 'G']:
        df_freq.loc[df_freq['ref_nucl'] == nucl, nucl] = np.nan
        

    for row in range(0, len(df_vcf)):
        mutation_pos = df_vcf.loc[row, 'POS'] - 1
        allele = genome[mutation_pos-1:mutation_pos+2]
        alt_nucl = df_vcf.loc[row, 'ALT']
        if (len(alt_nucl) != 1) or (len(df_vcf.loc[row, 'REF']) != 1) or (len(allele) != 3):
            continue

        ###Allele Depth (AD) in vcf file: first number - number of reads with ref nucl, second - with alt nucl (Nreads_alt)
        FORMAT = df_vcf.iloc[row, -2].split(':')
        AD_ind = FORMAT.index('AD')
        AF_ind = FORMAT.index('AF')
        AD = df_vcf.iloc[row, -1].split(':')[AD_ind]
        Nreads_alt = int(AD.split(',')[1])
        AF = df_vcf.iloc[row, -1].split(':')[AF_ind]
        freq = float(AF)
        if freq >= min_vf and freq < max_vf:
            df_Nreads.at[allele, alt_nucl] += Nreads_alt
            df_freq.at[allele, alt_nucl] += freq
        
    
    df_Nreads.reset_index(inplace=True)
    df_Nreads = df_Nreads.set_index(['allele'])
    df_Nreads = df_Nreads[['A', 'T', 'C', 'G']].stack()
    df_Nreads = df_Nreads.to_frame().reset_index()
    df_Nreads = df_Nreads.dropna()
    df_Nreads = df_Nreads.rename(columns={"level_1": "ALT", 0:'Nreads'})
    df_Nreads['REF'] = df_Nreads['allele'].apply(lambda x: x[1])
    df_Nreads['mutation'] = df_Nreads['REF'] + '>' + df_Nreads['ALT']
    df_Nreads = df_Nreads[['allele', 'REF', 'ALT', 'mutation', 'Nreads']]
    df_Nreads = df_Nreads.sort_values(by=['mutation', 'allele'])
    df_Nreads = df_Nreads.reset_index(drop=True)
    
    df_freq.reset_index(inplace=True)
    df_freq = df_freq.set_index(['allele'])
    df_freq = df_freq[['A', 'T', 'C', 'G']].stack()
    df_freq = df_freq.to_frame().reset_index()
    df_freq = df_freq.dropna()
    df_freq = df_freq.rename(columns={"level_1": "ALT", 0:'freq'})
    df_freq['REF'] = df_freq['allele'].apply(lambda x: x[1])
    df_freq['mutation'] = df_freq['REF'] + '>' + df_freq['ALT']
    df_freq = df_freq[['allele', 'REF', 'ALT', 'mutation', 'freq']]
    df_freq = df_freq.sort_values(by=['mutation', 'allele'])
    df_freq = df_freq.reset_index(drop=True)

    #make picture
    custom_palette = []
    for x in colors:
        custom_palette.extend([x]*16)
    ##divide dataframe into two parts
    df1_Nreads = df_Nreads[df_Nreads['REF'].isin(['A', 'C'])]
    df2_Nreads = df_Nreads[df_Nreads['REF'].isin(['G', 'T'])]
    df1_freq = df_freq[df_freq['REF'].isin(['A', 'C'])]
    df2_freq = df_freq[df_freq['REF'].isin(['G', 'T'])]

    mutation_list1 = df1_Nreads.mutation.to_list()
    mutation_list1 = list(set(mutation_list1))
    mutation_list1.sort()
    mutation_list2 = df2_Nreads.mutation.to_list()
    mutation_list2 = list(set(mutation_list2))
    mutation_list2.sort()

    ###picture for number of reads
    fig1 = plt.figure(figsize=(16, 7))
    ax11 = plt.subplot2grid(shape=(2, 1), loc=(0, 0))
    ax12 = plt.subplot2grid(shape=(2, 1), loc=(1, 0))

    ax11 = sns.barplot(x=df1_Nreads.index, y=df1_Nreads.Nreads, palette=custom_palette, ax=ax11)
    ax12 = sns.barplot(x=df2_Nreads.index, y=df2_Nreads.Nreads, palette=custom_palette, ax=ax12)
    ax11.set_xticklabels(df1_Nreads.allele, rotation=90, horizontalalignment='center')
    ax12.set_xticklabels(df2_Nreads.allele, rotation=90, horizontalalignment='center')

    ####change lim of picture
    top_position = int(df_Nreads.Nreads.max())
    round_pos = 0
    while top_position >= 10:
        top_position = int(top_position / 10)
        round_pos += 1
    new_top_position = round_up(df_Nreads.Nreads.max(), -round_pos)

    ####rectangle must place 10% of space
    part_ten = int(new_top_position/10)
    ax11.set_ylim(0, new_top_position+part_ten)
    ax12.set_ylim(0, new_top_position+part_ten)

    ####add rectangles
    for i in range(0, len(colors)):
        ax11.add_patch(Rectangle((i*16, new_top_position), 16, part_ten, facecolor = colors[i]))
        ax11.text(i*16+8, new_top_position+int(part_ten/2), mutation_list1[i], horizontalalignment='center', verticalalignment='center', size='x-large', color='white', weight='semibold')
        ax12.add_patch(Rectangle((i*16, new_top_position), 16, part_ten, facecolor = colors[i]))
        ax12.text(i*16+8, new_top_position+int(part_ten/2), mutation_list2[i], horizontalalignment='center', verticalalignment='center', size='x-large', color='white', weight='semibold')

    ####save picture
    fig1.suptitle('Number of reads by mutation type for sample '+ID+', '+feature_sample+'\naligned using '+method, fontsize=18)
    plt.tight_layout()
    plt.close(fig1)


    ###picture for freq
    fig2 = plt.figure(figsize=(16, 7))
    ax21 = plt.subplot2grid(shape=(2, 1), loc=(0, 0))
    ax22 = plt.subplot2grid(shape=(2, 1), loc=(1, 0))

    ax21 = sns.barplot(x=df1_freq.index, y=df1_freq.freq, palette=custom_palette, ax=ax21)
    ax22 = sns.barplot(x=df2_freq.index, y=df2_freq.freq, palette=custom_palette, ax=ax22)
    ax21.set_xticklabels(df1_freq.allele, rotation=90, horizontalalignment='center')
    ax22.set_xticklabels(df2_freq.allele, rotation=90, horizontalalignment='center')

    ####change lim of picture
    top_position = df_freq.freq.max()
    if top_position >= 1:
        new_top_position = top_position + 0.1
    else:
        round_pos = 0
        while top_position < 1:
            top_position = top_position * 10
            round_pos += 1
        new_top_position = round_up(df_freq.freq.max(), round_pos)
        

    ####rectangle must place 10% of space
    part_ten = new_top_position/10
    ax21.set_ylim(0, new_top_position+part_ten)
    ax22.set_ylim(0, new_top_position+part_ten)
    

    ####add rectangles
    for i in range(0, len(colors)):
        ax21.add_patch(Rectangle((i*16, new_top_position), 16, part_ten, facecolor = colors[i]))
        ax21.text(i*16+8, new_top_position+part_ten/2, mutation_list1[i], horizontalalignment='center', verticalalignment='center', size='x-large', color='white', weight='semibold')
        ax22.add_patch(Rectangle((i*16, new_top_position), 16, part_ten, facecolor = colors[i]))
        ax22.text(i*16+8, new_top_position+part_ten/2, mutation_list2[i], horizontalalignment='center', verticalalignment='center', size='x-large', color='white', weight='semibold')

    fig2.suptitle('Allele frequency by mutation type for sample '+ID+', '+feature_sample+'\naligned using '+method, fontsize=18)
    plt.tight_layout()
    plt.close(fig2)
    
    return [fig1, fig2]

In [25]:
# path to directory with vcf files
directory = r"\MPXV\clair3_vcf"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if f.endswith(".bam"):
        bam_path = f
        vcf_file = os.path.join(bam_path, "merge_output.vcf")
        sample_id = filename[:-4]
        
        #features from file 41597_2023_2149_MOESM2_ESM.xlsx
        feature = samples_features_dict[sample_id]
        feature = feature[5:]
        feature = feature.replace('_pass', '')
        print(sample_id, end=', ')
        plot_list = get_plot_vcf(vcf_file, sample_id, feature)
        for i in range(0, len(plot_list)):
            plot_list[i].savefig(directory+'/pictures/'+sample_id+'_'+feature+'_'+fig_list[i]+'.png')

ERR10513574, ERR10543479, ERR10543480, ERR10543481, ERR10550019, ERR10550020, ERR10550021, ERR10550022, ERR10550023, ERR10550024, ERR10550025, ERR10550026, ERR10550027, ERR10550028, ERR10550029, ERR10550030, ERR10550031, ERR10550032, ERR10550033, ERR10550034, ERR10550035, ERR10550036, ERR10963107, ERR10963108, ERR10963109, ERR10963110, ERR10963111, ERR10963112, ERR10963113, ERR10963114, ERR10963115, ERR10963116, ERR10963117, ERR10963118, ERR10963119, ERR10963120, ERR10963121, ERR10963122, ERR10963123, ERR10963124, ERR10963125, ERR10963126, ERR10963127, ERR10963128, 

Pictures are available https://drive.google.com/file/d/16i0P8gNPb4JL6YCDeZRfEDB77ym-wwaf/view?usp=sharing

## Comparison C>T mutations by motifs with other types of mutations

<a name='compare'></a>
[Return to Table of Contents](#TOC)

In [62]:
def my_filtering_function(pair):
    key, value = pair
    if "dRNA" in value or "mock" in value:
        return False  # keep pair in the filtered dictionary
    else:
        return True  # filter pair out of the dictionary

directory = r"\PRJEB56841"

#samples features
## table source - DOI: 10.1038/s41597-023-02149-4
samples_features = pd.read_excel(r"\41597_2023_2149_MOESM2_ESM.xlsx", skiprows=2)
samples_features = samples_features.fillna(method='ffill', axis=0)
samples_features.head()

samples_features_dict = samples_features.set_index('Run accession')['File names'].to_dict()

#filter mock and dRNA
filtered_samples_features = dict(filter(my_filtering_function, samples_features_dict.items()))
 
for key, value in filtered_samples_features.items():
    new_value = value.split('_')[2:]
    new_value[2] = new_value[2][5:]
    filtered_samples_features[key] = new_value
res = pd.DataFrame.from_dict(filtered_samples_features,orient='index', columns=['time', 'group', 'file_type'])
res['allele'] = ''
res.head()

,time,group,file_type,allele
ERR10550019,1h,A,bam,
ERR10963107,1h,A,fastq.gz,
ERR10550020,1h,B,bam,
ERR10963108,1h,B,fastq.gz,
ERR10550021,1h,C,bam,


In [7]:
##min and max for variant frequency
min_VF = 0.001
max_VF = 0.9

##read genome file
##to find position in genome: str(genome_dict['NC_045512.2'].seq)
genome_file = open(r"\NC_063383.1.fasta")
genome_dict = SeqIO.to_dict(SeqIO.parse(genome_file, "fasta"))

##mutations categories
category_list = ['TCT', 'TCA', 'TCC', 'TCG', '*']
category_complement_list = ['AGA', 'TGA', 'GGA', 'CGA', '*']

In [8]:
#function to rounding up top position
def round_up(n, decimals=0):
    multiplier = 10**decimals
    return math.ceil(n * multiplier) / multiplier

def Nread_freq_by_category(vcf_file, category = category_list, category_complement = category_complement_list, min_vf=min_VF, max_vf=max_VF, genome=str(genome_dict['NC_063383.1'].seq)):
    
    ##read vcf file
    ##skip description in vcf file
    with open(vcf_file, 'r') as f:
        reader=f.readlines()
        row = 0
        while reader[row].startswith('##') == True:
            row += 1
    df_vcf = pd.read_csv(vcf_file, skiprows=row, sep='\t')
    df_vcf = df_vcf.dropna()
    
    ##make dataframe from vcf file
    ###list with all allele variants of 3 nucleotides
    A = ['A'.join(i) for i in itertools.product('ATCG', repeat=2)]
    T = ['T'.join(i) for i in itertools.product('ATCG', repeat=2)]
    C = ['C'.join(i) for i in itertools.product('ATCG', repeat=2)]
    G = ['G'.join(i) for i in itertools.product('ATCG', repeat=2)]
    allele_list = A + T + C + G

    ###dataframe for counting reads number for mutations
    df_Nreads = pd.DataFrame(columns=['ref_nucl', 'allele', 'A', 'T', 'C', 'G'])
    df_Nreads['ref_nucl'] = ['A']*16 + ['T']*16 + ['C']*16 + ['G']*16
    df_Nreads['allele'] = allele_list
    df_Nreads = df_Nreads.set_index('allele')
    df_Nreads.fillna(0, inplace=True)
    ###replace cells with no mutation (like A>A) with NaN
    for nucl in ['A', 'T', 'C', 'G']:
        df_Nreads.loc[df_Nreads['ref_nucl'] == nucl, nucl] = np.nan
    
    ###dataframe for counting allele's frequencies for mutations
    df_freq = pd.DataFrame(columns=['ref_nucl', 'allele', 'A', 'T', 'C', 'G'])
    df_freq['ref_nucl'] = ['A']*16 + ['T']*16 + ['C']*16 + ['G']*16
    df_freq['allele'] = allele_list
    df_freq = df_freq.set_index('allele')
    df_freq.fillna(0, inplace=True)
    ###replace cells with no mutation (like A>A) with NaN
    for nucl in ['A', 'T', 'C', 'G']:
        df_freq.loc[df_freq['ref_nucl'] == nucl, nucl] = np.nan
        

    for row in range(0, len(df_vcf)):
        mutation_pos = df_vcf.loc[row, 'POS'] - 1
        allele = genome[mutation_pos-1:mutation_pos+2]
        alt_nucl = df_vcf.loc[row, 'ALT']
        if (len(alt_nucl) != 1) or (len(df_vcf.loc[row, 'REF']) != 1) or (len(allele) != 3):
            continue

        ###Allele Depth (AD) in vcf file: first number - number of reads with ref nucl, second - with alt nucl (Nreads_alt)
        FORMAT = df_vcf.iloc[row, -2].split(':')
        AD_ind = FORMAT.index('AD')
        AF_ind = FORMAT.index('AF')
        AD = df_vcf.iloc[row, -1].split(':')[AD_ind]
        Nreads_alt = int(AD.split(',')[1])
        AF = df_vcf.iloc[row, -1].split(':')[AF_ind]
        freq = float(AF)
        if freq >= min_vf and freq < max_vf:
            df_Nreads.at[allele, alt_nucl] += Nreads_alt
            df_freq.at[allele, alt_nucl] += freq
        
    
    df_Nreads.reset_index(inplace=True)
    df_Nreads = df_Nreads.set_index(['allele'])
    df_Nreads = df_Nreads[['A', 'T', 'C', 'G']].stack()
    df_Nreads = df_Nreads.to_frame().reset_index()
    df_Nreads = df_Nreads.dropna()
    df_Nreads = df_Nreads.rename(columns={"level_1": "ALT", 0:'Nreads'})
    df_Nreads['REF'] = df_Nreads['allele'].apply(lambda x: x[1])
    df_Nreads['mutation'] = df_Nreads['REF'] + '>' + df_Nreads['ALT']
    df_Nreads = df_Nreads[['allele', 'REF', 'ALT', 'mutation', 'Nreads']]
    df_Nreads = df_Nreads.sort_values(by=['mutation', 'allele'])
    df_Nreads = df_Nreads.reset_index(drop=True)
    
    #count for category
    Nreads = []
    for i in range(0, len(category[:-1])):
        motif = category[i]
        motif_complement = category_complement[i]
        Nr1 = df_Nreads[(df_Nreads['allele'] == motif) & (df_Nreads['mutation'] == 'C>T')]['Nreads'].sum()
        Nr2 = df_Nreads[(df_Nreads['allele'] == motif_complement) & (df_Nreads['mutation'] == 'G>A')]['Nreads'].sum()
        Nreads.append(Nr1+Nr2)
    Nreads.append(df_Nreads['Nreads'].sum() - sum(Nreads))
    
    df_freq.reset_index(inplace=True)
    df_freq = df_freq.set_index(['allele'])
    df_freq = df_freq[['A', 'T', 'C', 'G']].stack()
    df_freq = df_freq.to_frame().reset_index()
    df_freq = df_freq.dropna()
    df_freq = df_freq.rename(columns={"level_1": "ALT", 0:'freq'})
    df_freq['REF'] = df_freq['allele'].apply(lambda x: x[1])
    df_freq['mutation'] = df_freq['REF'] + '>' + df_freq['ALT']
    df_freq = df_freq[['allele', 'REF', 'ALT', 'mutation', 'freq']]
    df_freq = df_freq.sort_values(by=['mutation', 'allele'])
    df_freq = df_freq.reset_index(drop=True)
    
    #count for category
    freq = []
    for i in range(0, len(category[:-1])):
        motif = category[i]
        motif_complement = category_complement[i]
        freq1 = df_freq[(df_freq['allele'] == motif) & (df_freq['mutation'] == 'C>T')]['freq'].sum()
        freq2 = df_freq[(df_freq['allele'] == motif_complement) & (df_freq['mutation'] == 'G>A')]['freq'].sum()
        freq.append(freq1+freq2)
    freq.append(df_freq['freq'].sum() - sum(freq))
    
    new_list=[]
    for i in range(0, len(category)):
        new_list.append([category[i]]+[Nreads[i]]+[freq[i]])
    
    return new_list

In [87]:
vcf_dir = r"\clair3_vcf\vcf"

for key in filtered_samples_features:
    VCF_FILE = os.path.join(vcf_dir, key+'.vcf')
    res.loc[key]['allele'] = Nread_freq_by_category(VCF_FILE)

#transform table
transformed_res = res.explode('allele')
df2 = transformed_res.allele.apply(pd.Series)
df2.columns = ['allele', 'Nreads', 'freq']
result = pd.concat([transformed_res[transformed_res.columns[:-1]], df2], axis=1)
result.head(20)

,time,group,file_type,allele,Nreads,freq
ERR10550019,1h,A,bam,TCT,151.0,3.8339
ERR10550019,1h,A,bam,TCA,51.0,3.9896
ERR10550019,1h,A,bam,TCC,16.0,1.7451
ERR10550019,1h,A,bam,TCG,21.0,3.2864
ERR10550019,1h,A,bam,*,657.0,32.1077
ERR10963107,1h,A,fastq.gz,TCT,151.0,3.8339
ERR10963107,1h,A,fastq.gz,TCA,54.0,4.7229
ERR10963107,1h,A,fastq.gz,TCC,16.0,1.7451
ERR10963107,1h,A,fastq.gz,TCG,21.0,3.2864
ERR10963107,1h,A,fastq.gz,*,684.0,33.7102


In [90]:
outfile = os.path.join(r"\PRJEB56841\clair3_vcf", "summary_table_vcf.csv")
result.to_csv(outfile, sep='\t', index=True)

### for dRNA

In [4]:
def my_filtering_function(pair):
    key, value = pair
    if "dRNA" in value:
        return True  # keep pair in the filtered dictionary
    else:
        return False  # filter pair out of the dictionary

directory = r"\PRJEB56841"

#samples features
## table source - DOI: 10.1038/s41597-023-02149-4
samples_features = pd.read_excel(r"\PRJEB56841\41597_2023_2149_MOESM2_ESM.xlsx", skiprows=2)
samples_features = samples_features.fillna(method='ffill', axis=0)
samples_features.head()

samples_features_dict = samples_features.set_index('Run accession')['File names'].to_dict()

#filter mock and dRNA
filtered_samples_features = dict(filter(my_filtering_function, samples_features_dict.items()))

for key, value in filtered_samples_features.items():
    new_value = value.split('_')[3]
    new_value = [new_value[5:]]
    filtered_samples_features[key] = new_value
res = pd.DataFrame.from_dict(filtered_samples_features,orient='index', columns=['file_type'])
res['allele'] = ''
res.head()

{'ERR10513574': 'MPOX_dRNA_inf_pass.bam', 'ERR10963128': 'MPOX_dRNA_inf_pass.fastq.gz'}


,file_type,allele
ERR10513574,bam,
ERR10963128,fastq.gz,


In [9]:
vcf_dir = r"\PRJEB56841\clair3_vcf\vcf"

for key in filtered_samples_features:
    VCF_FILE = os.path.join(vcf_dir, key+'.vcf')
    res.loc[key]['allele'] = Nread_freq_by_category(VCF_FILE)

#transform table
transformed_res = res.explode('allele')
df2 = transformed_res.allele.apply(pd.Series)
df2.columns = ['allele', 'Nreads', 'freq']
result = pd.concat([transformed_res[transformed_res.columns[:-1]], df2], axis=1)
result.head(20)

,file_type,allele,Nreads,freq
ERR10513574,bam,TCT,35480.0,23.2629
ERR10513574,bam,TCA,8256.0,12.6355
ERR10513574,bam,TCC,1721.0,3.5774
ERR10513574,bam,TCG,11526.0,13.0403
ERR10513574,bam,*,13914.0,12.3461
ERR10963128,fastq.gz,TCT,34670.0,22.6684
ERR10963128,fastq.gz,TCA,8266.0,12.6514
ERR10963128,fastq.gz,TCC,1727.0,3.6897
ERR10963128,fastq.gz,TCG,11528.0,13.0198
ERR10963128,fastq.gz,*,15724.0,12.4549


In [10]:
outfile = os.path.join(r"\clair3_vcf", "summary_table_vcf_dRNA.csv")
result.to_csv(outfile, sep='\t', index=True)

Table with N reads and alelle frequence by motifs for dRNA samples is located in 
output_files/summary_table_vcf.csv

The tables are visualized using R - MPXV_vcf.Rmd

## Simulations

<a name='simulation'></a>
[Return to Table of Contents](#TOC)

### Visualization of shares of real and simulated positions with mutations from potential ones

<a name='shares'></a>
[Return to Table of Contents](#TOC)

In [ ]:
df = pd.read_csv('simulations_shares.csv', sep='\t', names=['time', 'replica', 'file_type', 'motif', '#simulation', 'N mutations', 'N mutations/N potential targets'], index_col=0)
df = df.reset_index()
df.head()

In [ ]:
# y - the proportion of the number of mutations in the genome of the number of potential mutations
colors_list = ['#C6878F', '#3D405B', '#81B29A', '#E07A5F', '#5F797B', '#F2CC8F']
generation_colors = ['#678e7b', '#74a08a', "#81b29a", '#8db9a4', '#9ac1ae', '#a6c9b8', '#b3d0c2', '#c0d8cc', '#cce0d6', '#d9e7e0']
my_palette = ['#E07A5F']+generation_colors

for file_type in ['fastq.gz', 'bam']:
    outfile = "simulations_shares_"+file_type+".png"
    df2 = df[df['file_type']==file_type]
    df2 = df2[df2['replica']!='dRNA']
    plot = sns.relplot(data=df2, x="time", y="N mutations/N potential targets",
                hue="#simulation", col="motif", col_order=['TCT', 'TCA', 'TCC', 'TCG'], row="replica",
                height=3, aspect=.7, kind="line", palette=my_palette, alpha=1)
    sns.set_style("whitegrid")
    plot.savefig(outfile, dpi=800) 

for dRNA

In [ ]:
# y - the proportion of the number of mutations in the genome of the number of potential mutations
colors_list = ['#C6878F', '#3D405B', '#81B29A', '#E07A5F', '#5F797B', '#F2CC8F']
generation_colors = ['#678e7b', '#74a08a', "#81b29a", '#8db9a4', '#9ac1ae', '#a6c9b8', '#b3d0c2', '#c0d8cc', '#cce0d6', '#d9e7e0'][::-1]
my_palette = ['#E07A5F']+generation_colors

for file_type in ['fastq.gz', 'bam']:
    outfile = "simulations_shares_dRNA_"+file_type+".png"
    df2 = df[df['file_type']==file_type]
    df2 = df2[df2['replica']=='dRNA']

    plt.figure(figsize=(5, 5))
    plot = sns.scatterplot(data=df2, x="motif", y="N mutations/N potential targets",
                hue="#simulation", palette=my_palette, legend=False)
    sns.set_style("whitegrid")
    fig = plot.get_figure()
    fig.savefig(outfile, dpi=800)

### Visualization of real and simulated number of reads for targets positions by replica and time

<a name='sim_Nreads'></a>
[Return to Table of Contents](#TOC)

process file simulations_Nreads_positions.csv

In [ ]:
import pandas as pd

def my_filtering_function(pair):
    key, value = pair
    if "mock" in value:
        return False  # filter pair out of the dictionary
    else:
        return True  # keep pair in the filtered dictionary

## motifs
motif_list = ['TCT', 'TCA', 'TCC', 'TCG']

## samples features
samples_features = pd.read_excel("41597_2023_2149_MOESM2_ESM.xlsx", skiprows=2)
samples_features = samples_features.fillna(method='ffill', axis=0)

samples_features_dict = samples_features.set_index('Run accession')['File names'].to_dict()

### filter mock
filtered_samples_features = dict(filter(my_filtering_function, samples_features_dict.items()))
samples_list = filtered_samples_features.keys()

### LIST WITH SAMPLES FEATURES
time_group_file_sample = dict()
for key, value in filtered_samples_features.items():
    new_value = value.split('_')[2:]
    try:
        new_value[2] = new_value[2][5:]
    except: #dRNA
        time = '-'
        group = 'dRNA'
        file_type = new_value[1][5:]
        new_feature = time+'_'+group+'_'+file_type
    else:
        new_feature = '_'.join(new_value)
    time_group_file_sample[key] = new_feature


# read dataframe with N reads for simulations
Nreads_pos_df = pd.read_csv('simulations_Nreads_positions.csv', sep=',', header=None)
Nreads_pos_df['sample_id'] = Nreads_pos_df.apply(lambda row: row[0].split("\t")[0], axis = 1)
Nreads_pos_df['motif'] = Nreads_pos_df.apply(lambda row: row[0].split("\t")[1], axis = 1)
Nreads_pos_df['feature'] = Nreads_pos_df.apply(lambda row: row[0].split("\t")[2], axis = 1)

pos_df = Nreads_pos_df[Nreads_pos_df['feature'] == 'targets_coordinates']
pos_df = pos_df.reset_index(drop=True)
Nreads_df = Nreads_pos_df[Nreads_pos_df['feature'] != 'targets_coordinates']
Nreads_df = Nreads_df.reset_index(drop=True)
for row in range(0, len(Nreads_df)):
    sample = Nreads_df.loc[row, "sample_id"]
    motif = Nreads_df.loc[row, "motif"]
    feature = Nreads_df.loc[row, "feature"]
    Nreads_list = Nreads_df.loc[row, 0].split("\t")
    Nreads_list = Nreads_list[3:]
    Nreads_list = list(map(int, Nreads_list))
    
    sample_features = time_group_file_sample[sample].split("_")
    time, group, file_type = sample_features[0], sample_features[1], sample_features[2]
    
    ## find targets_coordinates for these sample and motif
    pos_df_sample = pos_df[pos_df['sample_id'] == sample]
    pos_df_sample_motif = pos_df_sample[pos_df_sample['motif'] == motif]
    pos_df_sample_motif = pos_df_sample_motif.reset_index(drop=True)
    pos_list = pos_df_sample_motif.loc[0, 0].split("\t")
    pos_list = pos_list[3:]
    pos_list = list(map(int, pos_list))
    
    ## create new df for these sample, motif and feature and then combine with df
    new_dict = {'position': pos_list, 'motif': [motif]*len(pos_list),
                        'feature': [feature]*len(pos_list), 'Nreads':Nreads_list, 
                        'time': [time]*len(pos_list), 'group': [group]*len(pos_list), 
                        'file_type': [file_type]*len(pos_list)}
    df_new = pd.DataFrame(data=new_dict)
    df_new.to_csv("result.csv", sep='\t', index=False, mode='a', header=False)

plots

In [ ]:
colors_list = ['#C6878F', '#3D405B', '#81B29A', '#E07A5F', '#5F797B', '#F2CC8F']
def plot(df, file_type, my_palette=colors_list):
    plt.figure(figsize=(5, 5))
    df2 = df[df['file_type']==file_type]
    sns.set_style("whitegrid")
    plot = sns.relplot(data=df2, x="position", y="Nreads",
                hue="time", style="feature", col="replica", row="motif",
                height=3, aspect=1.2, kind="scatter", palette=my_palette)
    #plt.yscale('log')
    return plot

In [ ]:
df = pd.read_csv('result.csv', sep='\t', names=['position', 'motif', 'feature', 'Nreads', 'time', 'group', 'file_type'])
df = df[(df['feature']=='Nreads_mutated_sample') | (df['feature']=='Nreads_generation_1')]
df = df[df['Nreads']!=0]
df = df[df['group']!='dRNA']
df = df.replace('Nreads_mutated_sample', 'sample')
df = df.replace('Nreads_generation_1', 'simulation')
df =df.rename(columns={"group": "replica"})
df = df.reset_index(drop=True)

In [ ]:
for file_type in ['bam', 'fastq.gz']:
    f = plot(df, file_type)
    outfile = "simulations_Nreads_"+file_type+".png"
    f.savefig(outfile, dpi=800)

dRNA

In [ ]:
colors_list = ['#5F797B', '#F2CC8F']
def plot(df, file_type, my_palette=colors_list):
    plt.figure(figsize=(5, 5))
    df2 = df[df['file_type']==file_type]
    sns.set_style("whitegrid")
    plot = sns.relplot(data=df2, x="position", y="Nreads",
                hue="feature", col="motif", col_wrap=2,
                height=3, aspect=2, kind="scatter", palette=my_palette)
    return plot

In [ ]:
df = pd.read_csv('result.csv', sep='\t', names=['position', 'motif', 'feature', 'Nreads', 'time', 'group', 'file_type'])
df = df[(df['feature']=='Nreads_mutated_sample') | (df['feature']=='Nreads_generation_1')]
df = df[df['Nreads']!=0]
df = df[df['group']=='dRNA']
df = df.replace('Nreads_mutated_sample', 'sample')
df = df.replace('Nreads_generation_1', 'simulation')
df = df.reset_index(drop=True)

In [ ]:
for file_type in ['bam', 'fastq.gz']:
    f = plot(df, file_type)
    outfile = "simulations_Nreads_dRNA_"+file_type+".png"
    f.savefig(outfile, dpi=800)

### Get positions with the greatest and the least number of reads in real sample

<a name='greatest_pos'></a>
[Return to Table of Contents](#TOC)

#### Positions with the greatest number of reads in real sample

In [ ]:
df = pd.read_csv('result.csv', sep='\t', names=['position', 'motif', 'feature', 'Nreads', 'time', 'group', 'file_type'])
df = df[(df['feature']=='Nreads_mutated_sample') | (df['feature']=='Nreads_generation_1')]
df = df[df['group']!='dRNA']
df = df.replace('Nreads_mutated_sample', 'sample')
df = df.replace('Nreads_generation_1', 'simulation')
df =df.rename(columns={"group": "replica"})
df = df.reset_index(drop=True)

outfile = 'position_with_the_greatest_Nreads.csv'
with open(outfile, 'w') as fout:
    for file_type in ['bam', 'fastq.gz']:
        df_filetype = df[df['file_type']==file_type]
        fout.write(file_type+'\n')
        for motif in ['TCT', 'TCA', 'TCC', 'TCG']:
            df_motif = df_filetype[df_filetype['motif']==motif]
            fout.write(motif+'\n')
            for replica in ['A', 'B', 'C']:
                df_replica = df_motif[df_motif['replica']==replica]
                fout.write(replica+'\n')
                fout.write('position:Nreads_sample:Nreads_simulation\n')
                time_dict = dict()
                for time in ['1h', '2h', '4h', '6h', '12h', '24h']:
                    df_time = df_replica[df_replica['time']==time]
                    
                    df_sample = df_time[df_time['feature']=='sample']
                    df_simulation = df_time[df_time['feature']=='simulation']
                    df_simulation = df_simulation.reset_index(drop=True)
                    df_sample_sort = df_sample.sort_values(by='Nreads', ascending=False)
                    df_sample_sort = df_sample_sort.reset_index(drop=True)
                    
                    time_dict[time] = [df_sample_sort['position'].tolist()[:10], df_sample_sort['Nreads'].tolist()[:10]]
                fout.write('\t'.join(['1h', '2h', '4h', '6h', '12h', '24h'])+'\n')
                for i in range(0,10):
                    for time in ['1h', '2h', '4h', '6h', '12h', '24h']:
                        pos = time_dict[time][0][i]
                        Nreads_sample = time_dict[time][1][i]
                        pos_row_ind_simulation = df_simulation.index[df_simulation['position'] == pos]
                        Nreads_simulation = df_simulation.loc[pos_row_ind_simulation[0], 'Nreads']
                        fout.write(str(pos)+':'+str(Nreads_sample)+':'+str(Nreads_simulation))
                        if time == '24h':
                            fout.write('\n')
                        else:
                            fout.write('\t')
                fout.write('\n')

#### Positions with the greatest number of reads in simulated samples

In [ ]:
df = pd.read_csv('result.csv', sep='\t', names=['position', 'motif', 'feature', 'Nreads', 'time', 'group', 'file_type'])
df = df[(df['feature']=='Nreads_mutated_sample') | (df['feature']=='Nreads_generation_1')]
df = df[df['group']!='dRNA']
df = df.replace('Nreads_mutated_sample', 'sample')
df = df.replace('Nreads_generation_1', 'simulation')
df =df.rename(columns={"group": "replica"})
df = df.reset_index(drop=True)

outfile = 'position_with_the_least_Nreads.csv'
with open(outfile, 'w') as fout:
    for file_type in ['bam', 'fastq.gz']:
        df_filetype = df[df['file_type']==file_type]
        fout.write(file_type+'\n')
        for motif in ['TCT', 'TCA', 'TCC', 'TCG']:
            df_motif = df_filetype[df_filetype['motif']==motif]
            fout.write(motif+'\n')
            for replica in ['A', 'B', 'C']:
                df_replica = df_motif[df_motif['replica']==replica]
                fout.write(replica+'\n')
                fout.write('position:Nreads_sample:Nreads_simulation\n')
                time_dict = dict()
                for time in ['1h', '2h', '4h', '6h', '12h', '24h']:
                    df_time = df_replica[df_replica['time']==time]
                    
                    df_sample = df_time[df_time['feature']=='sample']
                    df_sample = df_sample.reset_index(drop=True)
                    df_simulation = df_time[df_time['feature']=='simulation']
                    
                    df_simulation_sort = df_simulation.sort_values(by='Nreads', ascending=False)
                    df_simulation_sort = df_simulation_sort.reset_index(drop=True)
                    
                    time_dict[time] = [df_simulation_sort['position'].tolist()[:20], df_simulation_sort['Nreads'].tolist()[:20]]
                fout.write('\t'.join(['1h', '2h', '4h', '6h', '12h', '24h'])+'\n')
                for i in range(0,20):
                    for time in ['1h', '2h', '4h', '6h', '12h', '24h']:
                        pos = time_dict[time][0][i]
                        Nreads_simulation = time_dict[time][1][i]
                        pos_row_ind_sample = df_sample.index[df_sample['position'] == pos]
                        Nreads_sample = df_sample.loc[pos_row_ind_sample[0], 'Nreads']
                        fout.write(str(pos)+':'+str(Nreads_sample)+':'+str(Nreads_simulation))
                        if time == '24h':
                            fout.write('\n')
                        else:
                            fout.write('\t')
                fout.write('\n')